In [ ]:
import numpy as np
import time
x=np.ones(20000000)*2
y=np.ones(20000000)*2
start=time.time()
result=np.sum(x*y)
end=time.time()
print("result is:",result)
print("time is:",(end-start))
result=0
start=time.time()
for i in range(20000000):
  result=result+x[i]*y[i]
end=time.time()
print("result is:",result)
print("time taken using for is:",(end-start))



result is: 80000000.0
time is: 0.09319424629211426
result is: 80000000.0
time taken using for is: 8.945921182632446


In [ ]:
#RDD and numpy can be combined for bulk computing
mat=np.random.rand(8,1)
print(mat)

[[0.08416663]
 [0.34023428]
 [0.95388069]
 [0.84471045]
 [0.52278157]
 [0.96671976]
 [0.26192486]
 [0.54170258]]


In [ ]:
mat=mat.reshape(4,-1)

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=576f5d05f34c470d8c2ace4772e5c160b5b92d5252555f781d87b14684261a1d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
sc=SparkContext.getOrCreate()
data_rdd=sc.parallelize(mat)
data_rdd.collect()
data_rdd.reduce(lambda x,y:np.add(x,y))



array([1.82275375, 2.69336707])

In [ ]:
#Vectorization
#Vectors.sparse or Vectors.dense
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Vector Demo").getOrCreate()
size=2
data=[(0,Vectors.dense(np.random.rand(size)),),
 (0,Vectors.dense(np.random.rand(size)),),
  (1,Vectors.dense(np.random.rand(size)),)]

In [ ]:
spark_data=spark.createDataFrame(data,["label","features"])
spark_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[0.56260353209659...|
|    0|[0.57584383225735...|
|    1|[0.50081402519395...|
+-----+--------------------+



In [ ]:
data=spark.read.options(mode="DROPMALFORMED").json("/content/iris.json")
data.show()
#convert df to rdd
iris_rdd=data.rdd.map(lambda x:[x[-1],Vectors.dense(x[:-1])])
iris_rdd.collect()

+-----------+----------+-----------+----------+-------+
|petalLength|petalWidth|sepalLength|sepalWidth|species|
+-----------+----------+-----------+----------+-------+
|        1.4|       0.2|        5.1|       3.5| setosa|
|        1.4|       0.2|        4.9|       3.0| setosa|
|        1.3|       0.2|        4.7|       3.2| setosa|
|        1.5|       0.2|        4.6|       3.1| setosa|
|        1.4|       0.2|        5.0|       3.6| setosa|
|        1.7|       0.4|        5.4|       3.9| setosa|
|        1.4|       0.3|        4.6|       3.4| setosa|
|        1.5|       0.2|        5.0|       3.4| setosa|
|        1.4|       0.2|        4.4|       2.9| setosa|
|        1.5|       0.1|        4.9|       3.1| setosa|
|        1.5|       0.2|        5.4|       3.7| setosa|
|        1.6|       0.2|        4.8|       3.4| setosa|
|        1.4|       0.1|        4.8|       3.0| setosa|
|        1.1|       0.1|        4.3|       3.0| setosa|
|        1.2|       0.2|        5.8|       4.0| 

[['setosa', DenseVector([1.4, 0.2, 5.1, 3.5])],
 ['setosa', DenseVector([1.4, 0.2, 4.9, 3.0])],
 ['setosa', DenseVector([1.3, 0.2, 4.7, 3.2])],
 ['setosa', DenseVector([1.5, 0.2, 4.6, 3.1])],
 ['setosa', DenseVector([1.4, 0.2, 5.0, 3.6])],
 ['setosa', DenseVector([1.7, 0.4, 5.4, 3.9])],
 ['setosa', DenseVector([1.4, 0.3, 4.6, 3.4])],
 ['setosa', DenseVector([1.5, 0.2, 5.0, 3.4])],
 ['setosa', DenseVector([1.4, 0.2, 4.4, 2.9])],
 ['setosa', DenseVector([1.5, 0.1, 4.9, 3.1])],
 ['setosa', DenseVector([1.5, 0.2, 5.4, 3.7])],
 ['setosa', DenseVector([1.6, 0.2, 4.8, 3.4])],
 ['setosa', DenseVector([1.4, 0.1, 4.8, 3.0])],
 ['setosa', DenseVector([1.1, 0.1, 4.3, 3.0])],
 ['setosa', DenseVector([1.2, 0.2, 5.8, 4.0])],
 ['setosa', DenseVector([1.5, 0.4, 5.7, 4.4])],
 ['setosa', DenseVector([1.3, 0.4, 5.4, 3.9])],
 ['setosa', DenseVector([1.4, 0.3, 5.1, 3.5])],
 ['setosa', DenseVector([1.7, 0.3, 5.7, 3.8])],
 ['setosa', DenseVector([1.5, 0.3, 5.1, 3.8])],
 ['setosa', DenseVector([1.7, 0.2, 5.4, 

In [ ]:
#cluster configuration
spark.stop()


In [ ]:
spark=SparkSession.builder.master("local[*]").config("spark.executor.memory","2g").config("spark.executor.cores",2).config("spark.shuffle.service.enabled","true").config("spark.dynamicAllocation.enabled","true").getOrCreate()

In [ ]:
from pyspark.ml.linalg import Vectors,Matrices
#create a vector Dense with value 12,45,67
data_dense=Vectors.dense([12,45,67])
print(data_dense)
data_sparse=Vectors.sparse(3,{0:1})
print(data_sparse)
dense_matrix=Matrices.dense(3,2,[22,21,33,34,35,35])
print(dense_matrix)
#colptr should be of size 3
#sparse(no.of rows,no.of cols,colptr,rowindex,data)
dense_sparse=Matrices.sparse(2,2,[0,1,2],[1,0],[2,3])
print(dense_sparse)

[12.0,45.0,67.0]
(3,[0],[1.0])
DenseMatrix([[22., 34.],
             [21., 35.],
             [33., 35.]])
2 X 2 CSCMatrix
(1,0) 2.0
(0,1) 3.0
